In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as panda
from tqdm import tqdm_notebook as tq
from selenium.webdriver.common.keys import Keys

In [17]:
driver = webdriver.Chrome(ChromeDriverManager().install())

In [30]:
driver.get("https://www.health.kr/main.asp")

driver.maximize_window()
time.sleep(1)

(driver.find_element_by_css_selector("#li_searchIngredient")).click()
time.sleep(1)
(driver.find_element_by_css_selector("button.search.ico")).click()
driver.implicitly_wait(10)
time.sleep(2)

english_names = []
korean_names = []
drugs = []

for i in tq(range(1, 292)):
    if i > 5:
        i = 6
    driver.find_element_by_css_selector(f"#paging > span > a:nth-child({i})").click()
    driver.implicitly_wait(10)
    time.sleep(1)
    
    ing_list = driver.find_elements_by_css_selector("td.name_eng")
    
    for j in range(len(ing_list)):
        drug_list = []
        english_names.append(driver.find_element_by_css_selector(f"tr:nth-child({j+2}) > td.name_eng").text)
        korean_names.append(driver.find_element_by_css_selector(f"tr:nth-child({j+2}) > td.name_kor").text)
        (driver.find_element_by_css_selector(f"tr:nth-child({j+2}) > td.product")).click()
        time.sleep(2)
        
        name_list = (driver.find_elements_by_css_selector("#proy_all > tbody > tr"))[1:]
        
        for k in range(len(name_list)):
            drug_list.append(driver.find_element_by_css_selector(f"#proy_all > tbody > tr:nth-child({k+2}) > td:nth-child(2)").text)
        
        drugs.append(", ".join(drug_list))
        
        driver.back()
        time.sleep(1)

C:\Users\user\AppData\Local\Temp\ipykernel_5364\4266485712.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tq(range(1, 292)):


  0%|          | 0/291 [00:00<?, ?it/s]

In [101]:
drugs_on_use = {"영어 성분명" : english_names, "한국어 성분명" : korean_names, "약품 리스트" : drugs}

In [102]:
import pandas as pd

In [103]:
drugs_on_use_in_korea = pd.DataFrame(drugs_on_use)

In [104]:
drugs_on_use_in_korea.to_excel("한국 의약품 리스트.xlsx", encoding = "UTF-8-sig")